In [1]:
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt

import plotly.graph_objects as go
from matplotlib.widgets import Slider, Button, RadioButtons
import matplotlib.gridspec as gridspec
import matplotlib.pyplot as plt
import cartopy.crs as ccrs
import cartopy.io.shapereader as shpreader
import plotly.offline as offline

import heapq

############################################
%matplotlib widget
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
import plotly.graph_objects as go
from matplotlib.widgets import Slider, Button, RadioButtons


import matplotlib.gridspec as gridspec
import matplotlib.pyplot as plt
import cartopy.crs as ccrs
import cartopy.io.shapereader as shpreader

import plotly as plotly
import plotly.offline as offline

import pandas as pd

import plotly

import plotly.graph_objs as go



import plotly.offline as offline

from plotly.graph_objs import *

from plotly.offline import download_plotlyjs, init_notebook_mode, plot, iplot


from collections import OrderedDict

init_notebook_mode(connected=True)

# class for storing election info for states in each year
class election_info:
    total_votes = 0
    
    # winner info
    winner_name = ''
    winner_votes = 0
    winner_party = ''
    
    # runnerup/loser info
    runnerup_name = ''
    runnerup_votes = 0
    runnerup_party = ''
    
    def __init__(self, winner_name, winner_votes, winner_party, total_votes, runnerup_name, runnerup_votes, runnerup_party):
        self.winner_name = winner_name
        self.winner_votes = winner_votes
        self.winner_party = winner_party
        self.total_votes = total_votes
        self.runnerup_name = runnerup_name
        self.runnerup_votes = runnerup_votes
        self.runnerup_party = runnerup_party
        

# read in election file
my_file = pd.read_csv('1976-2016-president.csv', encoding = 'unicode_escape')

# read in states and states abbreviation file
states_file = pd.read_csv('states.csv')
states_file = states_file.sort_values(by=['State'])

election_years = [1976, 1980, 1984, 1988, 1992, 1996, 2000, 2004, 2008, 2012, 2016]
states = list(states_file['State'])
state_codes = list(states_file['Abbreviation'])


# dict for storing years as key and then dict of states as vals
slides = {}
slides = dict((year,None) for year in election_years)
# create dict 

#for year in election_years:
#    slides[year] = None
#print(slides)
#print("states len", len(states))
#print(dict(states))


# go through each year 
for year in election_years:
    
    # create state dict for that year 
    slides[year] = OrderedDict((s,None) for s in states)
    #slides[year] = dict((s,None) for s in states)
    
    # get all data for that year
    year_info = my_file.loc[my_file['year'] == year]
    for state in states:
        state_info = year_info.loc[year_info['state'] == state]
        
        state_winner = state_info[state_info['candidatevotes']==state_info['candidatevotes'].max()]
        #print(state, "WINNER: ", state_winner)
        state_runnerup = state_info[state_info['candidatevotes']==heapq.nlargest(2, state_info['candidatevotes'])[1]]
        
        
        winner_name = str(state_winner['candidate'].values[0])
        winner_votes = int(state_winner['candidatevotes'].values[0])
        
        total_votes = int(state_winner['totalvotes'].values[0])
        winner_party = str(state_winner['party'].values[0])
        if(winner_party == 'democratic-farmer-labor'): party ='democrat'
        
        runnerup_name = str(state_runnerup['candidate'].values[0])
        runnerup_votes = int(state_runnerup['candidatevotes'].values[0])
        runnerup_party = str(state_runnerup['party'].values[0])
        if(runnerup_party == 'democratic-farmer-labor'): party ='democrat'
        
        #print(year, state, " WINNER: ", winner_name)
        #print(year, state, " WINNER RUNNERUP: ", runnerup_name)
        slides[year][state] = election_info(winner_name, winner_votes, winner_party, total_votes, runnerup_name, runnerup_votes, runnerup_party)   


#####################################################################
# Plot the US election popular vote slider from 1976-2016
data_slider = []

scl_cus = [[0.0, '#0015BC'],[1.0, '#DE0100']]

for key, val in slides.items():  
      
   
    data_one_year = dict(
                        type='choropleth',
                        locations = state_codes,
                        #z= [float(x.candidate_votes) for x in val.values()],
                        z = [0 if x.winner_party == 'democrat' else 1 for x in val.values()],
                        locationmode='USA-states',
                        #featureidkey = [ x.name for x in val.values()],
 
                        colorscale = scl_cus,
                        text =  [ str(str(x.winner_name)+ '<br>' + str(x.winner_party).title() + '<br>' + str(x.winner_votes) + " / " + str(x.total_votes) + " votes\n")  for x in val.values()],
                        showlegend = False,
                        showscale = False,
                        #hoverinfo = "location+text"
                        hovertemplate='%{location}<br>%{text}<extra></extra>'
                        )
    data_slider.append(data_one_year)



steps = []

for i in range(len(data_slider)):
    step = dict(method='restyle',
                args=['visible', [False] * len(data_slider)],
                label='Year {}'.format(i*4 + 1976)) # label to be displayed for each step (year)
    step['args'][1][i] = True
    steps.append(step)



##  I create the 'sliders' object from the 'steps' 

sliders = [dict(active=10, pad={"t": 1}, steps=steps)]  



layout = dict(title = 'US Popular Vote Election Map 1976-2016', geo=dict(scope='usa',
                       projection={'type': 'albers usa'}),
              sliders=sliders)
fig = dict(data=data_slider, layout=layout) 


plotly.offline.iplot(fig)
#offline.plot(fig, auto_open=True, image = 'png', image_filename="map_us_crime_slider" ,image_width=2000, image_height=1000, 
              #filename='map_us_crime_slider.html', validate=True)
#offline.plot(fig, auto_open=True, filename='tmp 1976-2016 US Election Map.html', validate=True)
offline.plot(fig, auto_open=True, filename='plots/1976-2016 US Election Map.html', validate=True)

#####################################################################


# winners for each election year
winners = {
    1976:"democrat",
    1980:"republican",
    1984:"republican",
    1988:"republican",
    1992:"democrat",
    1996:"democrat",
    2000:"republican",
    2004:"republican",
    2008:"democrat",
    2012:"democrat",
    2016:"republican" 
}

# count up total votes for election winner( via checking dem/rep), runnerup total, other votes total, and total votes
winner_totals = {}
runnerup_totals = {}
other_totals = {}
total_votes = {}
for YEAR, STATES in slides.items():
    winner_total = 0
    runnerup_total = 0
    other_total = 0
    tvc = 0
    for state, election_info in STATES.items():
        if(winners[YEAR] == "democrat" and election_info.winner_party == "democrat"):
            winner_total += election_info.winner_votes
            runnerup_total +=election_info.runnerup_votes
        elif(winners[YEAR] == "republican" and election_info.winner_party == "republican"):
            winner_total += election_info.winner_votes
            runnerup_total +=election_info.runnerup_votes
        else:
            winner_total +=election_info.runnerup_votes
            runnerup_total += election_info.winner_votes
            
        other_total += election_info.total_votes - election_info.winner_votes - election_info.runnerup_votes
        tvc+=election_info.total_votes
        
    winner_totals[YEAR]=winner_total
    runnerup_totals[YEAR] = runnerup_total
    other_totals[YEAR] = other_total
    total_votes[YEAR] = tvc
    

# plot 1 - Difference between winner and loser/runnerup of election over the years
import seaborn as sns
import matplotlib.pyplot as plt
axdf = pd.DataFrame()
#ax['year'] = election_years
axdf['diff'] = [winner_totals[i] - runnerup_totals[i] for i in election_years]
ax = sns.lineplot(data=axdf,color='b')
ax.set_xticks(range(len(axdf)))
ax.set_xticklabels(election_years)
legend = ax.legend()
legend.texts[0].set_text("Difference")
ax.set(xlabel='Year', ylabel='Difference')
ax.axhline(0, ls='--', color='r')
plt.title("Vote difference between winner and runnerup")
plt.plot(1)
plt.savefig('plots/1.png')
plt.show()


# plot 2 - plot percentage distribution of votes for each election year
fig = plt.figure(2)

ax1 = fig.add_subplot(341)
ax2 = fig.add_subplot(342)
ax3 = fig.add_subplot(343)
ax4 = fig.add_subplot(344)
ax5 = fig.add_subplot(345)
ax6 = fig.add_subplot(346)
ax7 = fig.add_subplot(347)
ax8 = fig.add_subplot(348)
ax9 = fig.add_subplot(349)
ax10 = fig.add_subplot(3,4,10)
ax11 = fig.add_subplot(3,4,11)
ax12 = fig.add_subplot(3,4,12)

#labels = 'winner', 'runnerup', 'other'
labels = 'Winner', 'Runnerup', 'Other'
colors = 'tab:blue', 'tab:orange', 'tab:green'

# colors=['tab:orange', 'tab:green', 'tab:purple']
sizes = [winner_totals[election_years[0]], runnerup_totals[election_years[0]], other_totals[election_years[0]]]
ax1.pie(sizes, labels=labels, autopct='%1.1f%%',
        shadow=True, startangle=90, textprops={'fontsize': 6}, colors=colors)
#ax1.pie(sizes,  autopct='%1.1f%%',
#        shadow=True, startangle=90, textprops={'fontsize': 6}, colors=['tab:purple', 'tab:orange', 'tab:green'])
ax1.set_title(election_years[0])


sizes = [winner_totals[election_years[1]], runnerup_totals[election_years[1]], other_totals[election_years[1]]]
ax2.pie(sizes, labels=labels, autopct='%1.1f%%',
        shadow=True, startangle=90, textprops={'fontsize': 6}, colors=colors)
ax2.set_title(election_years[1])

sizes = [winner_totals[election_years[2]], runnerup_totals[election_years[2]], other_totals[election_years[2]]]
ax3.pie(sizes, labels=labels, autopct='%1.1f%%',
        shadow=True, startangle=90, textprops={'fontsize': 6}, colors=colors)
ax3.set_title(election_years[2])

# set ax4 as legend, no a plot
handles, labels = ax1.get_legend_handles_labels()
ax4.axis('off')
ax4.legend(handles, labels,
          title="Votes",
         loc="lower right",
          bbox_to_anchor=(.5, 0, 0.5, .5))

sizes = [winner_totals[election_years[3]], runnerup_totals[election_years[3]], other_totals[election_years[3]]]
ax5.pie(sizes, labels=labels, autopct='%1.1f%%',
         shadow=True, startangle=90, textprops={'fontsize': 6}, colors=colors)
ax5.set_title(election_years[3])

sizes = [winner_totals[election_years[4]], runnerup_totals[election_years[4]], other_totals[election_years[4]]]
ax6.pie(sizes, labels=labels, autopct='%1.1f%%',
        shadow=True, startangle=90, textprops={'fontsize': 6}, colors=colors)
ax6.set_title(election_years[4])

sizes = [winner_totals[election_years[5]], runnerup_totals[election_years[5]], other_totals[election_years[5]]]
ax7.pie(sizes, labels=labels, autopct='%1.1f%%',
        shadow=True, startangle=90, textprops={'fontsize': 6}, colors=colors)
ax7.set_title(election_years[5])

sizes = [winner_totals[election_years[6]], runnerup_totals[election_years[6]], other_totals[election_years[6]]]
ax8.pie(sizes, labels=labels, autopct='%1.1f%%',
        shadow=True, startangle=90, textprops={'fontsize': 6}, colors=colors)
ax8.set_title(election_years[6])

sizes = [winner_totals[election_years[7]], runnerup_totals[election_years[7]], other_totals[election_years[7]]]
ax9.pie(sizes, labels=labels, autopct='%1.1f%%',
        shadow=True, startangle=90, textprops={'fontsize': 6}, colors=colors)
ax9.set_title(election_years[7])

sizes = [winner_totals[election_years[8]], runnerup_totals[election_years[8]], other_totals[election_years[8]]]
ax10.pie(sizes, labels=labels, autopct='%1.1f%%',
        shadow=True, startangle=90, textprops={'fontsize': 6}, colors=colors)
ax10.set_title(election_years[8])

sizes = [winner_totals[election_years[9]], runnerup_totals[election_years[9]], other_totals[election_years[9]]]
ax11.pie(sizes, labels=labels, autopct='%1.1f%%',
        shadow=True, startangle=90, textprops={'fontsize': 6}, colors=colors)
ax11.set_title(election_years[9])

sizes = [winner_totals[election_years[10]], runnerup_totals[election_years[10]], other_totals[election_years[10]]]
ax12.pie(sizes, labels=labels, autopct='%1.1f%%',
        shadow=True, startangle=90, textprops={'fontsize': 6}, colors=colors)
ax12.set_title(election_years[10])



fig.suptitle('Vote Distribution By Year', fontsize=16)

plt.tight_layout()
plt.plot(2)
plt.savefig('plots/2.png')
plt.show()


# plot 3 - Total Votes over the Years
fig = plt.figure(3)

axdf = pd.DataFrame()
axdf['Years'] = total_votes.keys()
axdf['Total Votes'] = total_votes.values()
ax = sns.barplot(data=axdf, x='Years', y='Total Votes', color='#f77f00')
plt.title("Total Votes Over Years")
plt.plot(3)
plt.savefig('plots/3.png')
plt.show()

# plot 4 - Total Winner votes over the years
fig = plt.figure(4)

axdf = pd.DataFrame()
axdf['Years'] = winner_totals.keys()
axdf['Winner Votes'] = winner_totals.values()
ax = sns.barplot(data=axdf, x='Years', y='Winner Votes', color='#f77f00')
plt.title("Winner's Votes Over Years")
plt.plot(4)
plt.savefig('plots/4.png')
plt.show()

# plot 5 - total runnerup votes over the years
fig = plt.figure(5)

axdf = pd.DataFrame()
axdf['Years'] = runnerup_totals.keys()
axdf['Runnerup Votes'] = runnerup_totals.values()
ax = sns.barplot(data=axdf, x='Years', y='Runnerup Votes', color='#f77f00')
plt.title("Runnerup's Votes Over Years")
plt.plot(5)
plt.savefig('plots/5.png')
plt.show()



# sum states vots over time for each party and classify them as rep or dem
from collections import OrderedDict 
state_leans = OrderedDict((state,[0,0]) for state in states)

for YEAR, STATES in slides.items():
    for state, election_info in STATES.items():
        if(election_info.winner_party == "democrat"):
            state_leans[state][0] += election_info.winner_votes
            state_leans[state][1] += election_info.runnerup_votes
        else:
            state_leans[state][0] += election_info.runnerup_votes
            state_leans[state][1] += election_info.winner_votes

# plot US map with state leanings from 1976 - 2016 (which party had more total votes in that time period)
data = dict(
                        type='choropleth',
                        locations = state_codes,
                        #z= [float(x.candidate_votes) for x in val.values()],
                        z = [0 if x[0] > x[1] else 1 for x in state_leans.values()],
                        locationmode='USA-states',
                        #featureidkey = [ x.name for x in val.values()],
 
                        colorscale = scl_cus,
                        text =  [ str('Democratic Lean <br>Dem: ' + str(x[0])+ ' votes' + '<br>' + "Rep: " + str(x[1]) + " votes") if x[0] > x[1] else str('Republican Lean <br>Rep: ' + str(x[1])+ ' votes' + '<br>' + "Dem: " + str(x[0]) + " votes")  for x in state_leans.values()],
                        showlegend = False,
                        showscale = False,
                        #hoverinfo = "location+text"
                        hovertemplate='%{location}<br>%{text}<extra></extra>'
                        )
    


layout = dict(
        title = 'State Leanings (1976-2016)',
        geo = dict(
            scope='usa',
            projection=dict( type='albers usa' ), 
            ),
             )
fig = dict( data=data, layout=layout )
plotly.offline.iplot(fig)
offline.plot(fig, auto_open=True, filename='plots/State Leanings.html', validate=True)



Canvas(toolbar=Toolbar(toolitems=[('Home', 'Reset original view', 'home', 'home'), ('Back', 'Back to previous …

Canvas(toolbar=Toolbar(toolitems=[('Home', 'Reset original view', 'home', 'home'), ('Back', 'Back to previous …

Canvas(toolbar=Toolbar(toolitems=[('Home', 'Reset original view', 'home', 'home'), ('Back', 'Back to previous …

Canvas(toolbar=Toolbar(toolitems=[('Home', 'Reset original view', 'home', 'home'), ('Back', 'Back to previous …

Canvas(toolbar=Toolbar(toolitems=[('Home', 'Reset original view', 'home', 'home'), ('Back', 'Back to previous …

'plots/State Leanings.html'